In [1]:
%run Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

Welcome to JupyROOT 6.29/01


In [2]:
isigma = 2
N_centr =5
N_hists = 2
centralities = ["0-88%","0-20%","20-40%","40-60%","60-88%"]
file_path="input/"
file_names="se-HeAu_18968_Artem_newcuts.root"
histo_names = ["r_min_pt_cent_distr","r_cnt_pt_cent_distr"]

In [3]:
hists_emc_read = []
infile = root.TFile.Open(file_path+file_names, "read")

for histname in histo_names:
    hists_emc_read.append(infile.Get(histname))
    hists_emc_read[-1].SetDirectory(root.nullptr)


hist_gamma   = infile.Get("gamma_pt_cent_distr")
hist_gamma.SetDirectory(root.nullptr)

infile.Close()

In [4]:
centrality = [1,-1,1,2,3,4,5,6,7,9]

pt = array( 'd' )
pt_boarders = array( 'd', [0.,.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,6.0,8.0,10.,20.])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append(0)
pt_boarders.append(20.)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)

pt =  array('d', [0.25, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.5, 7.0, 9.0, 15.0]) 
pt_boarders =  array('d', [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 6.0, 8.0, 10.0, 20.0, 20.0])


In [14]:
def emc_fitter(InHist, centr, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    N_land=1
    seed0 = seed
    seed=seed*N_pt

    hist_pt_bin = InHist.ProjectionY("kek")

    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}",hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1,centrality[2*centr],centrality[2*centr+1])
        #print("CENTRLITY :", centrality[2*centr], "-",centrality[2*centr+1])
        #print("PT :", pt_boarders[p], "-",pt_boarders[p+1])
        #print("bins :", pt_boarders[p], "-",pt_boarders[p+1])
        emc_var_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_land+3)*[2.] )
        g_land= root.TF1( f'g_land{seed+p}', 'landau',  0,3)
        g_pol = root.TF1( f'g_pol{seed+p}', '[0]+[1]*x+[2]*x*x',  4,10)
        Format_Func(g_land,3,4)
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}', 'landau(0)+[3]+[4]*x+[2]*x*x',0,10)
        Format_Func(total,2,1)


        if fits_mean!=0 and fits_sigma!=0:
            mean_ii=fits_sigma.Eval(pt[p])
            sigma_ii=fits_sigma.Eval(pt[p])
            g_land.SetParameter(1,sigma_ii)
            g_land.SetParameter(2,sigma_ii)
            g_land.SetParLimits(1,mean_ii*0.8-0.05,mean_ii*1.2+0.05)
            g_land.SetParLimits(2,sigma_ii*0.5,sigma_ii*1.5)
        #else:
        #    g_land.SetParameter(1,0.)
        #    g_land.SetParameter(2,1.)


        #emc_var_porj.GetXaxis().SetRangeUser(-1.4, 0.5);
        emc_var_porj.Draw()
        emc_var_porj.Fit( g_land, 'QR')
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_land = g_land.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_land[0], par_land[1], par_land[2], par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
                
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        par_land[0], par_land[1], par_land[2], par_pol[0], par_pol[1], par_pol[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]

        g_land.SetParameters(par_land)
        g_pol.SetParameters(par_pol)

        g_land.Draw('same')
        g_pol.Draw('same')

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        Mean_local_err.append(total.GetParError(1))
        Sigma_local_err.append(total.GetParError(2))

        bkg = total.Integral(3,6)

        int_fg_local.append(total.Integral(0,3) - bkg)
        int_bg_local.append(bkg)
            
        #m2_pos.Draw("same") m2_neg.Draw("same")
        m2_name="dist/"+str(seed0)+"/"+str(centr)+"/sigma_emc_"+str(round(pt[p],1))

        if fits_mean==0:
            c1.SaveAs( "output/" + m2_name + ".png" )
        else:
            c1.SaveAs( "output/" + m2_name + "upd.png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local]


In [15]:
def emc_centr_fitter(InHist, centralities, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    hist_ser = []
    for centr in centralities:
        hist_ser.append(emc_fitter(InHist, centr, seed, sigma, fits_mean, fits_sigma))
    return hist_ser

In [16]:
import multiprocess as mp

Ntr = N_hists

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read[i], [0,1,2,3,4], i, isigma) for i in range(Ntr)])
pool.close()

In [17]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG = [], [], [], [], [], []
for i in range(Ntr):
    Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0 = [], [], [], [], [], []
    for j in range(N_centr):
        Means0.append(output_array[i][j][0])
        Mean_errs0.append(output_array[i][j][1])
        Sigmas0.append(output_array[i][j][2])
        Sigma_errs0.append(output_array[i][j][3])
        Int_FG0.append(output_array[i][j][4])
        Int_BG0.append(output_array[i][j][5])
        for ier in range(len(Sigma_errs0[j])):
            Sigma_errs0[j][ier]+=Sigmas0[j][ier]*0.05
            Mean_errs0[j][ier]+=abs(Means0[j][ier])*0.10
    Means.append(Means0)
    Mean_errs.append(Mean_errs0)
    Sigmas.append(Sigmas0)
    Sigma_errs.append(Sigma_errs0)
    Int_FG.append(Int_FG0)
    Int_BG.append(Int_BG0)

In [18]:
with open('output/sMean.txt', 'w') as f:
    for mean_centr in Means:
        for mean_part in mean_centr:
            f.write(f"{mean_part}\n")

with open('output/sSigma.txt', 'w') as f:
    for sigma_centr in Sigmas:
        for sigma_part in sigma_centr:
            f.write(f"{sigma_part}\n")

In [19]:
types = ["emc_r_sdphi_","emc_min_r_sdphi_","emc_cnt_sdphi_","emc_r_sdz_","emc_min_r_sdz_","emc_cnt_sdz_"]
colors=[1,2,4,6,root.kGreen+2]
fit_functions, fit_functions0= [], []
for itype in range(N_hists):
    fit_functions_loc, fit_functions_loc0  = [], []
    for icentr in range(N_centr):
        fit_functions_loc0.append(root.TF1("fit_s_sigma_loc_functions_"+types[itype]+centralities[icentr],"0*([0]*[6])+([1]+[2]*x+[3]*x*x*x+[4]*x*x*x*x+[5]*sqrt(x))",0.25,20))
        fit_functions_loc.append(root.TF1("fit_s_sigma_functions_"+types[itype]+centralities[icentr],"exp(-[0]*x)*([1]+[2]*x+[3]*x*x*x+[4]*x*x*x*x+[5]*sqrt(x))+[6]",0.25,20))
        fit_functions_loc[-1].SetLineColor(colors[icentr])
        fit_functions_loc[-1].SetLineWidth(3)
    fit_functions.append(fit_functions_loc)
    fit_functions0.append(fit_functions_loc0)

In [22]:
h2=[]
c3 = root.TCanvas(f"c3",f"c3",500*2,500*1)
c3.Divide(2,1)
g_sigma = [[[]]*N_centr]*N_hists
g_sigma_inter = [[[]]*6]*N_hists
maxes = [2,2,2,2,2,2,2]
names = ["emc_r_sdphi","emc_min_r_sdphi","emc_cnt_sdphi","emc_r_sdz","emc_min_r_sdz","emc_cnt_sdz"]
legends =[]
for itype in range(N_hists):
    c3.cd(itype+1)
    h2.append(Format_Pad_old( 0.05, 16, 0, maxes[itype], "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.03, 0.03, "" ))
    h2[-1].Draw()
    legends.append(root.TLegend(0.3,0.5,0.7,0.9,names[itype]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for icentr in range(0,N_centr):
        
        g_sigma[itype][icentr].append(root.TGraphErrors(N_pt, pt, Sigmas[itype][icentr],pt_err,Sigma_errs[itype][icentr]))
        g_sigma_inter[itype][icentr].append(root.TGraphErrors(N_pt))

        g_sigma[itype][icentr][-1].AddPoint(0.5,0.5*(Sigmas[itype][icentr][0]+Sigmas[itype][icentr][1]))
        g_sigma[itype][icentr][-1].SetPointError(N_pt,0,Sigma_errs[itype][icentr][0])
        
        Format_Graph(g_sigma[itype][icentr][-1],20,1, colors[icentr], 3, colors[icentr],1,1)
        Format_Graph(g_sigma_inter[itype][icentr][-1], 21, 0, colors[icentr], 3, colors[icentr], 1, 1)
        g_sigma[itype][icentr][-1].Fit(fit_functions0[itype][icentr],"Q","Q",0.25,3.25)
        fit_functions[itype][icentr].SetParameters(fit_functions0[itype][icentr].GetParameters())
        fit_functions[itype][icentr].SetParameter(0,8)
        fit_functions[itype][icentr].SetParLimits(0,0.4,16)
        g_sigma[itype][icentr][-1].Fit(fit_functions[itype][icentr],"Q","")
        g_sigma[itype][icentr][-1].Draw('P')
        #g_sigma_inter[itype][icentr][-1].Draw('sameL')
        legends[-1].AddEntry(g_sigma[itype][icentr][-1],centralities[icentr],"p")
        legends[-1].AddEntry(fit_functions[itype][icentr],f"fit","l")
        
    legends[-1].Draw()
c3.Draw()
c3.SaveAs("output/r_emc_sigma.png")

In [23]:
#output_file = root.TFile.Open("output/HeAu_emcid_s_sigmas.root","RECREATE")
#for fit_functions_syst in fit_functions:
#    for fits in fit_functions_syst: fits.Write()
#output_file.Close()

In [24]:
fit_mean_functions0,  fit_mean_functions = [], []
for itype in range(N_hists):
    fit_mean_functions_loc, fit_mean_functions_loc0 = [], []
    for icentr in range(N_centr):
        fit_mean_functions_loc0.append(root.TF1("fit_s_mean_loc_functions_"+types[itype]+centralities[icentr],"0*([0]*[6])+([1]+[2]*x+[3]*x*x*x+[4]*x*x*x*x+[5]*sqrt(x))",0.25,20))
        fit_mean_functions_loc.append(root.TF1("fit_s_mean_functions_"+types[itype]+centralities[icentr],"exp(-[0]*x)*([1]+[2]*x+[3]*x*x*x+[4]*x*x*x*x+[5]*sqrt(x))+[6]",0.25,20))
        fit_mean_functions_loc[-1].SetLineWidth(3)
        fit_mean_functions_loc[-1].SetLineColor(colors[icentr])

    fit_mean_functions.append(fit_mean_functions_loc)
    fit_mean_functions0.append(fit_mean_functions_loc0)

In [28]:
h3=[]
c4 = root.TCanvas(f"c4",f"c4",500*2,500*1)
c4.Divide(2,1)
g_mean = [[[]]*N_centr]*N_hists
maxes = [2,2,0.2,0.2,0.2,0.2]
names = ["emc_r_sdphi","emc_min_r_sdphi","emc_cnt_sdphi","emc_r_sdz","emc_min_r_sdz","emc_cnt_sdz"]
legends =[]
for itype in range(N_hists):
    c4.cd(itype+1)
    h3.append(Format_Pad_old( 0., 16, 0, maxes[itype], "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.03, 0.03, "" ))
    h3[-1].Draw()
    legends.append(root.TLegend(0.3,0.5,0.7,0.9,names[itype]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for icentr in range(0,N_centr):

        g_mean[itype][icentr].append(root.TGraphErrors(N_pt, pt, Means[itype][icentr],pt_err,Mean_errs[itype][icentr]))
        Format_Graph(g_mean[itype][icentr][-1],20,1, colors[icentr], 3, colors[icentr],1,1)
        g_mean[itype][icentr][-1].AddPoint(0.5,0.5*(Means[itype][icentr][0]+Means[itype][icentr][1]))
        g_mean[itype][icentr][-1].SetPointError(N_pt,0,Sigma_errs[itype][icentr][0])
        g_mean[itype][icentr][-1].AddPoint(0.3125,(Means[itype][icentr][0]*0.75+Means[itype][icentr][1]*0.25))
        g_mean[itype][icentr][-1].SetPointError(N_pt+1,0,Sigma_errs[itype][icentr][0])
        g_mean[itype][icentr][-1].Fit(fit_mean_functions0[itype][icentr],"Q","Q",0.25,3.25)
        fit_mean_functions[itype][icentr].SetParameters(fit_mean_functions0[itype][icentr].GetParameters())
        fit_mean_functions[itype][icentr].SetParameter(0,4)
        fit_mean_functions[itype][icentr].SetParLimits(0,0.5,4)
        g_mean[itype][icentr][-1].Fit("pol0","Q","Q",2.,5.25)
        if itype!=2 and itype!=5: fit_mean_functions[itype][icentr].FixParameter(6,g_mean[itype][icentr][-1].GetFunction("pol0").GetParameter(0))
        g_mean[itype][icentr][-1].Fit(fit_mean_functions[itype][icentr],"Q","")
        #params_mean_sigma[syst][centr][0].append(all_fit_sigma_low[ii].GetParameters())
        g_mean[itype][icentr][-1].Draw('P')
        legends[-1].AddEntry(g_mean[itype][icentr][-1],centralities[icentr],"p")
        legends[-1].AddEntry(fit_mean_functions[itype][icentr],f"fit","l")
        
    legends[-1].Draw()
c4.Draw()
c4.SaveAs("output/r_emc_mean.png")

In [29]:
#func_names_to_print=[]
#func_formula_to_print=[]
#func_params_to_print=[]
#
#output_file_mean = root.TFile.Open("output/HeAu_s_emcid.root","RECREATE")
#for fit_functions_syst in fit_functions:
#    for fits in fit_functions_syst: 
#        fits.Write()
#        func_names_to_print.append(fits.GetName())
#        func_formula_to_print.append(fits.GetExpFormula())
#        kek = []
#        for i in range(fits.GetNpar()):
#            kek.append(fits.GetParameter(i))
#        func_params_to_print.append(kek)
#for fit_functions_centr in fit_mean_functions:
#    for mean_fits in fit_functions_centr: 
#        mean_fits.Write()
#        func_names_to_print.append(mean_fits.GetName())
#        func_formula_to_print.append(mean_fits.GetExpFormula())
#        kek = []
#        for i in range(mean_fits.GetNpar()):
#            kek.append(mean_fits.GetParameter(i))
#        func_params_to_print.append(kek)
#output_file_mean.Close()
#if False: print(func_names_to_print, "\n", func_formula_to_print, "\n", func_params_to_print)

In [46]:
bw_dphi_hist = hists_emc_read[0].ProjectionX("keks")
bw_dphi = bw_dphi_hist.GetBinWidth(1)
bw_dz_hist = hists_emc_read[1].ProjectionX("kekz")
bw_dz = bw_dz_hist.GetBinWidth(1)

print(bw_dphi,bw_dz)

FG_BG, FG_BG_err = [], []
for icase in range(14):
    FG_BG0, FG_BG_err0 = [], []
    for icentr in range(N_centr):
        hist_gamma_proj = hist_gamma.ProjectionY(f"gamma{icentr}",1,1,centrality[2*icentr],centrality[2*icentr+1])
        FG_BG_loc =  array( 'd' )
        FG_BG_err_loc =  array( 'd' )
        for ipt in range(N_pt):
            gamma_yield = 0.1
            for j in range(hist_gamma_proj.FindBin(pt_boarders[ipt]),hist_gamma_proj.FindBin(pt_boarders[ipt+1])):
                gamma_yield += hist_gamma_proj.GetBinContent(j)
                
            if icase == 0:
                FG_BG_loc.append(Int_FG[0][icentr][ipt]/(Int_BG[0][icentr][ipt]+Int_FG[0][icentr][ipt]))
                FG_BG_err_loc.append((abs(Int_FG[0][icentr][ipt]))**0.5/(Int_BG[0][icentr][ipt]+Int_FG[0][icentr][ipt])*bw_dphi**0.5)
            if icase == 1:
                FG_BG_loc.append(Int_FG[1][icentr][ipt]/(Int_BG[1][icentr][ipt]+Int_FG[1][icentr][ipt]))
                FG_BG_err_loc.append((abs(Int_FG[1][icentr][ipt]))**0.5/(Int_BG[1][icentr][ipt]+Int_FG[1][icentr][ipt])*bw_dz**0.5)
            if icase == 2:
                FG_BG_loc.append((Int_BG[0][icentr][ipt]+Int_FG[0][icentr][ipt])/bw_dphi/gamma_yield)
                FG_BG_err_loc.append((abs(Int_FG[0][icentr][ipt]/bw_dphi))**0.5/gamma_yield)
            if icase == 3:
                FG_BG_loc.append((Int_BG[1][icentr][ipt]+Int_FG[1][icentr][ipt])/bw_dz/gamma_yield)
                FG_BG_err_loc.append((abs(Int_FG[1][icentr][ipt]/bw_dz))**0.5/gamma_yield)
            if icase == 4:
                FG_BG_loc.append((Int_FG[0][icentr][ipt])/bw_dphi/gamma_yield)
                FG_BG_err_loc.append((abs(Int_FG[0][icentr][ipt]/bw_dphi))**0.5/gamma_yield)
            if icase == 5:
                FG_BG_loc.append((Int_FG[1][icentr][ipt])/bw_dz/gamma_yield)
                FG_BG_err_loc.append((abs(Int_FG[1][icentr][ipt]/bw_dz))**0.5/gamma_yield)
            if icase == 6:
                FG_BG_loc.append((Int_BG[0][icentr][ipt])/bw_dphi/gamma_yield)
                FG_BG_err_loc.append((abs((Int_BG[0][icentr][ipt]+Int_FG[0][icentr][ipt])/bw_dphi))**0.5/gamma_yield)
            if icase == 7:
                FG_BG_loc.append((Int_BG[1][icentr][ipt])/bw_dz/gamma_yield)
                FG_BG_err_loc.append((abs(Int_BG[1][icentr][ipt]/bw_dz))**0.5/gamma_yield)
            if icase == 8:
                FG_BG_loc.append(1-(1-(Int_BG[0][icentr][ipt])/bw_dphi/gamma_yield)**2)
                FG_BG_err_loc.append(2*(abs((Int_BG[0][icentr][ipt]+Int_FG[0][icentr][ipt])/bw_dphi))**0.5/gamma_yield)
            if icase == 9:
                FG_BG_loc.append(1-(1-(Int_BG[1][icentr][ipt])/bw_dphi/gamma_yield)**2)
                FG_BG_err_loc.append(2*(abs((Int_BG[1][icentr][ipt]+Int_FG[1][icentr][ipt])/bw_dphi))**0.5/gamma_yield)
        FG_BG0.append(FG_BG_loc)
        FG_BG_err0.append(FG_BG_err_loc)
    FG_BG.append(FG_BG0)
    FG_BG_err.append(FG_BG_err0)

0.1 0.1


In [47]:
h5=[]
c5 = root.TCanvas(f"c5",f"c5",500*2,500*4)
c5.Divide(2,4)
g_mean = [[[]]*12]*12
maxes = [1,1,0.2,0.2,0.2,0.2,0.05,0.05]
names = ["2#sigma d#phi FG/(FG+BG)","2#sigma dz FG/(FG+BG)","2#sigma both","2#sigma d#phi (FG+BG)/N_{#gamma}","2#sigma dz (FG+BG)/N_{#gamma}","2#sigma both",\
         "2#sigma d#phi FG/N_{#gamma}","2#sigma dz FG/N_{#gamma}","2#sigma both","2#sigma d#phi BG/N_{#gamma}","2#sigma dz BG/N_{#gamma}","2#sigma both"]
legends =[]
for itype in range(8):
    c5.cd(itype+1)
    h5.append(Format_Pad_old( 0.25, 10, 0, maxes[itype], "p_{T} (GeV/#it{c})", "Probability", 1.1, 1.1, 0.03, 0.03, "" ))
    h5[-1].Draw()
    legends.append(root.TLegend(0.4,0.6,0.9,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetTextAlign(23)
    legends[-1].SetLineWidth(0)
    legends[-1].SetTextSize(0.05)
    for icentr in range(0,N_centr):

        g_mean[itype][icentr].append(root.TGraphErrors(N_pt, pt, FG_BG[itype][icentr],pt_err,FG_BG_err[itype][icentr]))
        Format_Graph(g_mean[itype][icentr][-1], 20, 1, colors[icentr], 3, colors[icentr], 1, 1)

        g_mean[itype][icentr][-1].Draw('P')
        legends[-1].AddEntry(g_mean[itype][icentr][-1],names[itype]+" "+centralities[icentr],"p")
        
    legends[-1].Draw()
c5.Draw()
c5.SaveAs("output/s_emc_prob.png")

In [60]:
h6=[]
c6 = root.TCanvas(f"c6",f"c6",500*2,500*2)
c6.Divide(2,2)
g_prob = [[[]]*12]*12
ptmax = [16,16,32,32]
maxes = [0.1,0.1,0.5,0.5]
names = ["2#sigma d#phi BG/N_{#gamma}","2#sigma dz BG/N_{#gamma}","2#sigma d#phi BG/N_{#gamma}","2#sigma dz BG/N_{#gamma}"]
legends =[]
pt_eta = array('d', [0.25, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.5, 7.0, 9.0, 15.0])
for i in range(len(pt_eta)):
    pt_eta[i]*=2
print(pt)
for itype in range(4):
    c6.cd(itype+1)
    h6.append(Format_Pad_old( 0.3, ptmax[itype], 0, maxes[itype], "p_{T} (GeV/#it{c})", "Probability for #eta", 1.1, 1.1, 0.03, 0.03, "" ))
    h6[-1].Draw()
    legends.append(root.TLegend(0.4,0.6,0.9,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetTextAlign(23)
    legends[-1].SetLineWidth(0)
    legends[-1].SetTextSize(0.05)
    for icentr in range(0,N_centr):

        g_prob[itype][icentr].append(root.TGraphErrors(N_pt, pt_eta, FG_BG[itype%2+8][icentr],pt_err,FG_BG_err[itype%2+8][icentr]))
        Format_Graph(g_prob[itype][icentr][-1], 20, 1, colors[icentr], 3, colors[icentr], 1, 1)

        g_prob[itype][icentr][-1].Draw('P')
        legends[-1].AddEntry(g_prob[itype][icentr][-1],names[itype]+" "+centralities[icentr],"p")
        
    legends[-1].Draw()
c6.Draw()
c6.SaveAs("output/s_emc_prob_sigma.png")

array('d', [4.0, 12.0, 20.0, 28.0, 36.0, 44.0, 52.0, 60.0, 68.0, 76.0, 88.0, 112.0, 144.0, 240.0, 4.0, 12.0, 20.0, 28.0, 36.0, 44.0, 52.0, 60.0, 68.0, 76.0, 88.0, 112.0, 144.0, 240.0, 4.0, 12.0, 20.0, 28.0, 36.0, 44.0, 52.0, 60.0, 68.0, 76.0, 88.0, 112.0, 144.0, 240.0, 4.0, 12.0, 20.0, 28.0, 36.0, 44.0, 52.0, 60.0, 68.0, 76.0, 88.0, 112.0, 144.0, 240.0, 4.0, 12.0, 20.0, 28.0, 36.0, 44.0, 52.0, 60.0, 68.0, 76.0, 88.0, 112.0, 144.0, 240.0, 4.0, 12.0, 20.0, 28.0, 36.0, 44.0, 52.0, 60.0, 68.0, 76.0, 88.0, 112.0, 144.0, 240.0, 4.0, 12.0, 20.0, 28.0, 36.0, 44.0, 52.0, 60.0, 68.0, 76.0, 88.0, 112.0, 144.0, 240.0, 4.0, 12.0, 20.0, 28.0, 36.0, 44.0, 52.0, 60.0, 68.0, 76.0, 88.0, 112.0, 144.0, 240.0, 4.0, 12.0, 20.0, 28.0, 36.0, 44.0, 52.0, 60.0, 68.0, 76.0, 88.0, 112.0, 144.0, 240.0, 4.0, 12.0, 20.0, 28.0, 36.0, 44.0, 52.0, 60.0, 68.0, 76.0, 88.0, 112.0, 144.0, 240.0, 4.0, 12.0, 20.0, 28.0, 36.0, 44.0, 52.0, 60.0, 68.0, 76.0, 88.0, 112.0, 144.0, 240.0, 4.0, 12.0, 20.0, 28.0, 36.0, 44.0, 52.0, 6

In [19]:
!jupyter nbconvert --to html emc_r_HeAu.ipynb --HTMLExporter.theme=dark;

[NbConvertApp] Converting notebook emc_s_sigmas_HeAu.ipynb to html
/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)
[NbConvertApp] Writing 1876210 bytes to emc_s_sigmas_HeAu.html
